<a href="https://colab.research.google.com/github/rajunod/cookbooks_various/blob/main/NOAA_climo_percentiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workflow for displaying Percentiles from CPC

The material in this notebook is licensed for free and open consumption and resuse. All code is served under